In [885]:
# Para no tener que reiniciar el kernel
%load_ext autoreload
%autoreload 2
import sys  # System-specific parameters and functions
sys.path.append("../")  # Adding the parent directory to the system path
from tqdm import tqdm
from src import sp_nlp as sn
import pandas as pd
# Para trabajar con Mongo
# ------------------------------------------------------------------------------
import pymongo


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


from bs4 import BeautifulSoup
import requests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [886]:
cliente = pymongo.MongoClient()
# nos conectamos con una BBDD 
imdb = cliente['IMDB']
peliculas = imdb['Peliculas']
actores = imdb['Actores']
api = imdb['Api']
nlp = imdb['NLP']

In [887]:
df =pd.DataFrame(nlp.find())

In [889]:
# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(df["argumento"])

# Calcular la matriz de similitud de coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [891]:
id_IMDB = peliculas.find_one({'id_IMDB':api.find_one({'title': 'Batman Begins'})['id']})['id_IMDB']

In [892]:
def recomendar_peliculas(titulo, df, cosine_sim):
    # Obtener el índice de la película en el DataFrame
    id_IMDB = peliculas.find_one({'id_IMDB':api.find_one({'title': titulo})['id']})['id_IMDB']
    idx = df[df["id_IMDB"] == id_IMDB].index[0]
    
    # Obtener las puntuaciones de similitud con todas las películas
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar las películas por similitud (de mayor a menor)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los índices de las 5 películas más similares (excluyendo la original)
    top_peliculas = sim_scores[1:6]



    # Retornar los títulos de las películas recomendadas
    return df.iloc[[i[0] for i in top_peliculas]]["id_IMDB"].values


In [893]:
id_pelis = recomendar_peliculas("Batman Begins", df, cosine_sim)
list(api.find({'id':{'$in':list(id_pelis)}},{"title": 1,"_id": 0}))

[{'title': 'Batman: Vengeance'},
 {'title': 'Bat-Family'},
 {'title': 'Getting Up: Contents Under Pressure'},
 {'title': 'Batman Unlimited: Animal Instincts'},
 {'title': 'The Penguin'}]

In [894]:
id_pelis = recomendar_peliculas("Avengers", df, cosine_sim)
list(api.find({'id':{'$in':list(id_pelis)}},{"title": 1,"_id": 0}))

[{'title': 'Krypto the Superdog'},
 {'title': 'Wonder Man'},
 {'title': 'X-Men 2: Clone Wars'},
 {'title': 'Tottoko Hamutaro'},
 {'title': 'Young Ones'}]

In [895]:
id_pelis = recomendar_peliculas("The Dark Avengers", df, cosine_sim)
list(api.find({'id':{'$in':list(id_pelis)}},{"title": 1,"_id": 0}))

[{'title': 'The Knock'},
 {'title': 'Aeterna. Part 1'},
 {'title': 'Two Nights'},
 {'title': 'Si Doel the Movie 2'},
 {'title': 'Bad Connection'}]

In [896]:
id_pelis = recomendar_peliculas("Free Willy", df, cosine_sim)
list(api.find({'id':{'$in':list(id_pelis)}},{"title": 1,"_id": 0}))

[{'title': 'Onna Senshi Efe and Jelia: Goude no monsho'},
 {'title': 'Thru the Moebius Strip'},
 {'title': 'Io non ho paura'},
 {'title': 'Host'},
 {'title': 'The Light'}]

In [897]:
id_pelis = recomendar_peliculas("Mortadelo y Filemón contra Jimmy el Cachondo", df, cosine_sim)
list(api.find({'id':{'$in':list(id_pelis)}},{"title": 1,"_id": 0}))

[{'title': 'Hard Rain'},
 {'title': '20 Funerals'},
 {'title': 'Attack Force'},
 {'title': 'K.V.'},
 {'title': 'Zombie Massacre'}]

In [898]:
id_pelis = recomendar_peliculas("Tarzan", df, cosine_sim)
list(api.find({'id':{'$in':list(id_pelis)}},{"title": 1,"_id": 0}))

[{'title': 'The Legend of Tarzan'},
 {'title': 'Cura koja je voljela bajke'},
 {'title': 'Without Paris'},
 {'title': 'Tur'},
 {'title': 'An lian tao hua yuan'}]

In [899]:
id_pelis = recomendar_peliculas("Cars 4", df, cosine_sim)
list(api.find({'id':{'$in':list(id_pelis)}},{"title": 1,"_id": 0}))

TypeError: 'NoneType' object is not subscriptable

In [793]:
list(api.find({"type": "Movie", 'genre':['Animation']}))

[{'_id': ObjectId('679b98610c8ad3cc3b33cf15'),
  'id': 'tt0368305',
  'title': 'Subdivision, Colorado',
  'type': 'Movie',
  'genre': ['Animation'],
  'year': 2004},
 {'_id': ObjectId('679b986b0c8ad3cc3b3431c1'),
  'id': 'tt0099472',
  'title': 'Ducktales de film: Het geheim van de wonderlamp',
  'type': 'Movie',
  'genre': ['Animation'],
  'year': 1990},
 {'_id': ObjectId('679b986b0c8ad3cc3b3431c4'),
  'id': 'tt0099410',
  'title': 'Despertaferro',
  'type': 'Movie',
  'genre': ['Animation'],
  'year': 1990},
 {'_id': ObjectId('679b986b0c8ad3cc3b3431cb'),
  'id': 'tt0100477',
  'title': 'The Rescuers Down Under',
  'type': 'Movie',
  'genre': ['Animation'],
  'year': 1990},
 {'_id': ObjectId('679b986b0c8ad3cc3b3431ce'),
  'id': 'tt0100281',
  'title': 'The Nutcracker Prince',
  'type': 'Movie',
  'genre': ['Animation'],
  'year': 1990},
 {'_id': ObjectId('679b986c0c8ad3cc3b3444bc'),
  'id': 'tt0374248',
  'title': 'Strings',
  'type': 'Movie',
  'genre': ['Animation'],
  'year': 2004}

In [481]:
peliculas.find_one({'id_IMDB':api.find_one({'title': 'Bugs'})['id']})

{'_id': ObjectId('679817c7ff0fb55006e40913'),
 'id_IMDB': 'tt0111904',
 'puntuacion': 6.9,
 'direccion': ['Brian Clemens'],
 'guion': ['Jaye Griffiths', 'Jesse Birdsall', 'Craig McLachlan'],
 'duracion': 50,
 'argumento': 'Ed, Ros, Alex and Beckett form a high-tech crime-fighting team working for the mysterious and secretive head of Bureau 2, Jan, and are only assigned the most difficult and dangerous cases.',
 'actores': []}